# Dataset Creation for N-gram Language Model

**from** Yibarek Tadesse

**Course:** GenAI for Software Development (CSCI 455/555)  
**Date:** February 2026  
**Instructor:** Dr. Antonio Mastropaolo  
**TA:** Md. Zahidul Haque Alvi

---
## Setup & Imports

In [1]:
# Install required packages
!pip install javalang gitpython pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import subprocess
import json
import random
import shutil
import stat
import time
from pathlib import Path

import pandas as pd
import javalang
from javalang.tokenizer import tokenize

BASE_DIR = Path(".") / "dataset"
CLONE_DIR = BASE_DIR / "java_repos"
OUTPUT_DIR = BASE_DIR / "ngram_dataset"

random.seed(42)

CLASSES_PER_REPO = 10   # Java files to sample per repo
MIN_TOKENS = 10         # Minimum tokens per method

VAL_SIZE = 1000
TEST_SIZE = 1000
T1_CAP = 15000
T2_CAP = 25000
T3_CAP = 35000

# Create directories if they do not exist

CLONE_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Setup complete!")
print(f"Clone directory: {CLONE_DIR.resolve()}")
print(f"Output directory: {OUTPUT_DIR.resolve()}")

Setup complete!
Clone directory: C:\Users\creep\Downloads\n_gram_model\dataset\java_repos
Output directory: C:\Users\creep\Downloads\n_gram_model\dataset\ngram_dataset


---
## Fetch Repository List


In [ ]:
import requests
import pandas as pd

def fetch_top_java_repos(num_repos=500, per_page=100):
    """
    Fetch top-starred Java repositories from GitHub API.
    Filters applied:
    - language:java (Fixed language requirement)
    - stars:>1000 (Popularity)
    - size:>10000 (Size > 10MB)
    - pushed:>2025-01-01 (Recent activity)
    - archived:false (Exclusion rule)
    - topic:leetcode/interview (Avoid impractical, isolated code)
    - fork=False (Exclusion rule to avoid duplicate code)
    """
    repos = []
    page = 1

    while len(repos) < num_repos:
        url = "https://api.github.com/search/repositories"
        
        # Query string with defined criteria
        params = {
            "q": "language:java stars:>1000 size:>10000 pushed:>2025-01-01 archived:false -topic:leetcode -topic:interview",
            "sort": "stars",
            "order": "desc",
            "per_page": per_page,
            "page": page
        }

        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        data = response.json()
        items = data.get("items", [])

        if not items:
            break

        for item in items:
            # Explicitly exclude forks to prevent data duplication
            if item.get("fork", False):
                continue

            repos.append({
                "full_name": item["full_name"],
                "clone_url": item["clone_url"],
                "stars": item["stargazers_count"],
                "size_kb": item["size"],
                "last_pushed": item["pushed_at"],
                "description": item.get("description", "")
            })

        page += 1

        if len(repos) >= num_repos:
            break

    return repos[:num_repos]

# Fetch repositories
print("Fetching top Java repositories from GitHub...")
repo_data = fetch_top_java_repos(num_repos=500)
df_repos = pd.DataFrame(repo_data)

print(f"\nFetched {len(df_repos)} repositories")
print(f"\nTop 10 repos by stars:")
print(df_repos)

Fetching top Java repositories from GitHub...

Fetched 500 repositories

Top 10 repos by stars:
                                full_name  \
0           iluwatar/java-design-patterns   
1                         macrozheng/mall   
2             spring-projects/spring-boot   
3                     doocs/advanced-java   
4                   elastic/elasticsearch   
..                                    ...   
495                     WhatsApp/stickers   
496              bytedeco/javacpp-presets   
497                       assertj/assertj   
498  sqlcipher/android-database-sqlcipher   
499                     apache/cloudstack   

                                             clone_url  stars  size_kb  \
0    https://github.com/iluwatar/java-design-patter...  93757    47229   
1               https://github.com/macrozheng/mall.git  82927    58726   
2    https://github.com/spring-projects/spring-boot...  80050   211658   
3           https://github.com/doocs/advanced-java.git  78839    26

In [ ]:
# Standard Windows installation path for Git
GIT_EXE = r"C:\Program Files\Git\cmd\git.exe"

def remove_readonly(func, path, excinfo):
    """Error handler for shutil.onexc to fix Access Denied errors."""
    try:
        os.chmod(path, stat.S_IWRITE)
        func(path)
    except Exception:
        pass # Ignore if we still can't delete it

def clone_repo(clone_url, dest_dir, max_retries=3):
    """
    Shallow clone a repository with long-path support and error visibility.
    """
    for attempt in range(1, max_retries + 1):
        try:
            # Clean up directory from a previous failed attempt
            if os.path.exists(dest_dir):
                shutil.onexc(dest_dir, onerror=remove_readonly)

            git_cmd = GIT_EXE if os.path.exists(GIT_EXE) else "git"
            
            # Used to bypass 260-char limits
            cmd = f'"{git_cmd}" clone -c core.longpaths=true --depth 1 --quiet {clone_url} "{dest_dir}"'
            
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=240)

            if result.returncode == 0:
                return True
            else:
                print(f"  [Attempt {attempt}] Failed. Git says: {result.stderr.strip()}")
                
        except subprocess.TimeoutExpired:
            print(f"  [Attempt {attempt}] Timeout exceeded...")
        except Exception as e:
            print(f"  [Attempt {attempt}] Python Error: {e}")
            
        # Brief pause before retrying to give API limit breathing room
        if attempt < max_retries:
            time.sleep(3)

    return False

# Clone repositories
cloned_repos = []
failed_repos = []


print(f"Cloning repositories ...\n")

for idx, row in df_repos.iterrows():
    repo_name = row["full_name"]
    clone_url = row["clone_url"]

    safe_name = repo_name.replace("/", "_")
    dest_dir = os.path.join(CLONE_DIR, safe_name)

    print(f"[{idx+1}/{len(df_repos)}] Cloning {repo_name}...", end=" ")

    success = clone_repo(clone_url, dest_dir)

    if success:
        cloned_repos.append({
            "repo_name": repo_name,
            "local_path": dest_dir,
            "stars": row["stars"]
        })
        print("done")
    else:
        failed_repos.append(repo_name)
        print("failed")

print(f"\nSummary:")
print(f"  Successfully cloned: {len(cloned_repos)}")
print(f"  Failed: {len(failed_repos)}")

Cloning repositories ...

[1/500] Cloning iluwatar/java-design-patterns... done
[2/500] Cloning macrozheng/mall... done
[3/500] Cloning spring-projects/spring-boot... done
[4/500] Cloning doocs/advanced-java... done
[5/500] Cloning elastic/elasticsearch... done
[6/500] Cloning NationalSecurityAgency/ghidra... done
[7/500] Cloning spring-projects/spring-framework... done
[8/500] Cloning google/guava... done
[9/500] Cloning dbeaver/dbeaver... done
[10/500] Cloning ReactiveX/RxJava... done
[11/500] Cloning skylot/jadx... done
[12/500] Cloning jeecgboot/JeecgBoot... done
[13/500] Cloning apache/dubbo... done
[14/500] Cloning PhilJay/MPAndroidChart... done
[15/500] Cloning halo-dev/halo... done
[16/500] Cloning eugenp/tutorials... done
[17/500] Cloning TeamNewPipe/NewPipe... done
[18/500] Cloning alibaba/arthas... done
[19/500] Cloning airbnb/lottie-android... done
[20/500] Cloning YunaiV/ruoyi-vue-pro... done
[21/500] Cloning bumptech/glide... done
[22/500] Cloning netty/netty... done
[23/

---
## Find and Select Java Files


In [5]:
def find_java_files(repo_path):
    """
    Find all .java files in a repository.
    Excludes test files and common non-source directories.
    """
    java_files = []
    exclude_patterns = ["test", "tests", "example", "examples", "sample", "demo", "generated"]

    for root, dirs, files in os.walk(repo_path):
        root_lower = root.lower()
        if any(pattern in root_lower for pattern in exclude_patterns):
            continue

        for file in files:
            if file.endswith(".java"):
                java_files.append(os.path.join(root, file))

    return java_files


def select_java_files(java_files, max_files):
    """
    Randomly select up to max_files from the list.
    """
    if len(java_files) <= max_files:
        return java_files
    return random.sample(java_files, max_files)


# Find and select Java files from each repo
repo_java_files = {}
all_selected_files = []

print(f"Finding Java files (selecting up to {CLASSES_PER_REPO} per repo)...\n")

for repo_info in cloned_repos:
    repo_name = repo_info["repo_name"]
    repo_path = repo_info["local_path"]

    java_files = find_java_files(repo_path)

    if not java_files:
        print(f"  {repo_name}: No Java files found")
        continue

    selected = select_java_files(java_files, max_files=CLASSES_PER_REPO)

    repo_java_files[repo_name] = {
        "total_files": len(java_files),
        "selected_files": [os.path.relpath(f, repo_path) for f in selected],
        "remaining_files": len(java_files) - len(selected)
    }

    all_selected_files.extend([(repo_name, f) for f in selected])
    print(f"  {repo_name}: {len(selected)}/{len(java_files)} files selected")

print(f"\nTotal Java files selected: {len(all_selected_files)}")

Finding Java files (selecting up to 10 per repo)...

  iluwatar/java-design-patterns: 10/1262 files selected
  macrozheng/mall: 10/508 files selected
  spring-projects/spring-boot: 10/3721 files selected
  doocs/advanced-java: 1/1 files selected
  elastic/elasticsearch: 10/14081 files selected
  NationalSecurityAgency/ghidra: 10/13281 files selected
  spring-projects/spring-framework: 10/4929 files selected
  google/guava: 10/1270 files selected
  dbeaver/dbeaver: 10/5986 files selected
  ReactiveX/RxJava: 10/914 files selected
  skylot/jadx: 10/1209 files selected
  jeecgboot/JeecgBoot: 10/786 files selected
  apache/dubbo: 10/2390 files selected
  PhilJay/MPAndroidChart: 10/158 files selected
  halo-dev/halo: 10/896 files selected
  eugenp/tutorials: 10/14951 files selected
  TeamNewPipe/NewPipe: 10/290 files selected
  alibaba/arthas: 10/756 files selected
  airbnb/lottie-android: 10/215 files selected
  YunaiV/ruoyi-vue-pro: 10/3548 files selected
  bumptech/glide: 10/396 files sel

---
## Parse and Extract Methods

Here, we:
1. Parse the source code using `javalang`
2. Extract all method declarations
3. Store the method body along with metadata (repo, file, method name)

In [ ]:
import os

def read_file_content(file_path):
    """Read file content with multiple encoding fallbacks and Windows long-path support."""
    
    if os.name == 'nt':
        abs_path = os.path.abspath(file_path)
        if not abs_path.startswith('\\\\?\\'):
            file_path = '\\\\?\\' + abs_path

    encodings = ['utf-8', 'latin-1', 'cp1252']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                return f.read()
        except UnicodeDecodeError:
            continue
        except FileNotFoundError:
            return None
        except Exception:
            return None

    return None


def extract_method_source(source_code, method_node, lines):
    """Extract the source code of a method by counting braces."""
    try:
        start_line = method_node.position.line - 1

        brace_count = 0
        started = False
        end_line = start_line

        for i in range(start_line, len(lines)):
            line = lines[i]
            for char in line:
                if char == '{':
                    brace_count += 1
                    started = True
                elif char == '}':
                    brace_count -= 1

            if started and brace_count == 0:
                end_line = i
                break

        method_lines = lines[start_line:end_line + 1]
        return '\n'.join(method_lines)

    except Exception:
        return None


def extract_methods_from_file(file_path, repo_name):
    """Parse a Java file and extract all methods."""
    methods = []

    source_code = read_file_content(file_path)
    if source_code is None:
        return methods

    lines = source_code.split('\n')

    try:
        tree = javalang.parse.parse(source_code)

        for path, node in tree.filter(javalang.tree.MethodDeclaration):
            method_source = extract_method_source(source_code, node, lines)

            if method_source:
                methods.append({
                    "repo": repo_name,
                    "file": os.path.basename(file_path),
                    "method_name": node.name,
                    "source": method_source
                })

    except javalang.parser.JavaSyntaxError:
        pass
    except Exception:
        pass

    return methods


# Extract methods from all selected files
all_methods = []

print(f"Extracting methods from {len(all_selected_files)} files...\n")

for i, (repo_name, file_path) in enumerate(all_selected_files):
    if (i + 1) % 100 == 0:
        print(f"  Processed {i + 1}/{len(all_selected_files)} files...")

    methods = extract_methods_from_file(file_path, repo_name)
    all_methods.extend(methods)

print(f"\nTotal methods extracted: {len(all_methods)}")

Extracting methods from 4810 files...

  Processed 100/4810 files...
  Processed 200/4810 files...
  Processed 300/4810 files...
  Processed 400/4810 files...
  Processed 500/4810 files...
  Processed 600/4810 files...
  Processed 700/4810 files...
  Processed 800/4810 files...
  Processed 900/4810 files...
  Processed 1000/4810 files...
  Processed 1100/4810 files...
  Processed 1200/4810 files...
  Processed 1300/4810 files...
  Processed 1400/4810 files...
  Processed 1500/4810 files...
  Processed 1600/4810 files...
  Processed 1700/4810 files...
  Processed 1800/4810 files...
  Processed 1900/4810 files...
  Processed 2000/4810 files...
  Processed 2100/4810 files...
  Processed 2200/4810 files...
  Processed 2300/4810 files...
  Processed 2400/4810 files...
  Processed 2500/4810 files...
  Processed 2600/4810 files...
  Processed 2700/4810 files...
  Processed 2800/4810 files...
  Processed 2900/4810 files...
  Processed 3000/4810 files...
  Processed 3100/4810 files...
  Process

---
## Filter Methods

Methods that contain non-ASCII characters and fewer than 10 tokens are removed.

In [7]:
def contains_non_ascii(text):
    """Check if text contains non-ASCII characters."""
    try:
        text.encode('ascii')
        return False
    except UnicodeEncodeError:
        return True


def count_tokens(source_code):
    """Count the number of Java tokens in source code."""
    try:
        tokens = list(tokenize(source_code))
        return len(tokens)
    except:
        return 0


# Apply filters
filtered_methods = []
stats = {
    "total": len(all_methods),
    "non_ascii_dropped": 0,
    "too_short_dropped": 0,
    "kept": 0
}

print(f"Filtering {len(all_methods)} methods...\n")

for method in all_methods:
    source = method["source"]

    # 1. Remove non-ASCII characters
    if contains_non_ascii(source):
        stats["non_ascii_dropped"] += 1
        continue

    # 2. Filter methods with fewer than MIN_TOKENS (10 tokens)
    token_count = count_tokens(source)
    if token_count < MIN_TOKENS:
        stats["too_short_dropped"] += 1
        continue

    method["token_count"] = token_count
    filtered_methods.append(method)
    stats["kept"] += 1

print(f"Filtering Results:")
print(f"  Total methods:        {stats['total']}")
print(f"  Dropped (non-ASCII):  {stats['non_ascii_dropped']}")
print(f"  Dropped (< {MIN_TOKENS} tokens): {stats['too_short_dropped']}")
print(f"  -------------------------")
print(f"  Methods kept:         {stats['kept']}")

Filtering 39866 methods...

Filtering Results:
  Total methods:        39866
  Dropped (non-ASCII):  1181
  Dropped (< 10 tokens): 2186
  -------------------------
  Methods kept:         36499


---
## Tokenize Methods

Methods are tokenized space-separated tokens using `javalang.tokenizer` such that they look like:

```
public void setName ( String name ) { this . name = name ; }
```

In [20]:
def tokenize_method(source_code):
    """Tokenize Java source code into space-separated tokens."""
    try:
        tokens = list(tokenize(source_code))
        token_values = [token.value for token in tokens]
        return ' '.join(token_values)
    except:
        return None


# Tokenize all methods
tokenized_methods = []

print(f"Tokenizing {len(filtered_methods)} methods...\n")

for method in filtered_methods:
    tokenized = tokenize_method(method["source"])

    if tokenized:
        tokenized_methods.append({
            "repo": method["repo"],
            "file": method["file"],
            "method_name": method["method_name"],
            "tokenized_code": tokenized,
            "token_count": method["token_count"]
        })

print(f"Successfully tokenized: {len(tokenized_methods)} methods")

print(f"\nExample tokenized method:")
if tokenized_methods:
    example = tokenized_methods[0]
    print(f"  Repo: {example['repo']}")
    print(f"  File: {example['file']}")
    print(f"  Method: {example['method_name']}")
    print(f"  Tokens ({example['token_count']}):")
    print(f"  {example['tokenized_code'][:200]}..." if len(example['tokenized_code']) > 200 else f"  {example['tokenized_code']}")

Tokenizing 36499 methods...

Successfully tokenized: 36499 methods

Example tokenized method:
  Repo: iluwatar/java-design-patterns
  File: App.java
  Method: main
  Tokens (70):
  public static void main ( String [ ] args ) { var injector = Guice . createInjector ( new LotteryTestingModule ( ) ) ; var administration = injector . getInstance ( LotteryAdministration . class ) ; a...


---
## Clean, Deduplicate, and Split

Coinduct further removal by finding methods:
- With more than one method signatures in one line
- That are not complete (not ending with `}`)

In addition, duplicate methods are removed and methods are split into train/val/test sets.

Finally, vocabs are created from scratch and <UNK> mapping implementation is set up for training stages.




In [21]:
def is_clean_method(tokenized_code):
    """Check if method is clean (single method, complete)."""
    method_keywords = tokenized_code.count("public ") + tokenized_code.count("private ") + tokenized_code.count("protected ")
    if method_keywords > 1:
        return False
    if not tokenized_code.endswith("}"):
        return False
    return True

# Clean
print(f"Before cleaning: {len(tokenized_methods)}")
tokenized_methods = [m for m in tokenized_methods if is_clean_method(m['tokenized_code'])]
print(f"After cleaning: {len(tokenized_methods)}")

# Deduplicate
seen = set()
unique_methods = []
for m in tokenized_methods:
    if m['tokenized_code'] not in seen:
        seen.add(m['tokenized_code'])
        unique_methods.append(m)

print(f"After dedup: {len(unique_methods)}")
tokenized_methods = unique_methods

# Shuffle and split
random.shuffle(tokenized_methods)

val_size = VAL_SIZE
test_size = TEST_SIZE
train_size = len(tokenized_methods) - val_size - test_size

# 1. Base Splits
train_data_full = tokenized_methods[:train_size]
val_data = tokenized_methods[train_size:train_size + val_size]
test_data = tokenized_methods[train_size + val_size:train_size + val_size + test_size]

# 2. Create T1, T2, T3 based on assignment caps
T1 = train_data_full[:min(len(train_data_full), T1_CAP)]  # Capped at 15,000
T2 = train_data_full[:min(len(train_data_full), T2_CAP)]  # Capped at 25,000
T3 = train_data_full[:min(len(train_data_full), T3_CAP)]  # Capped at 35,000

print(f"\nDataset Splits:")
print(f"  T1 (capped at {T1_CAP}): {len(T1)} methods")
print(f"  T2 (capped at {T2_CAP}): {len(T2)} methods")
print(f"  T3 (capped at {T3_CAP}): {len(T3)} methods")
print(f"  Validation: {len(val_data)} methods")
print(f"  Test:       {len(test_data)} methods")

# 3. Build separate vocabularies from scratch for T1, T2, T3
def build_vocab(dataset):
    """Builds a vocabulary set of unique tokens from a given dataset."""
    vocab = set()
    for m in dataset:
        tokens = m['tokenized_code'].split()
        vocab.update(tokens)
    return vocab

vocab_T1 = build_vocab(T1)
vocab_T2 = build_vocab(T2)
vocab_T3 = build_vocab(T3)

print(f"\nVocabularies Built:")
print(f"  Vocab T1 size: {len(vocab_T1)} unique tokens")
print(f"  Vocab T2 size: {len(vocab_T2)} unique tokens")
print(f"  Vocab T3 size: {len(vocab_T3)} unique tokens")

# 4. Implement <UNK> Mapping for Validation/Testing
def apply_unk_mapping(dataset, vocab):
    """Maps tokens not present in the corresponding vocabulary to <UNK>."""
    mapped_dataset = []
    for m in dataset:
        tokens = m['tokenized_code'].split()
        # If token is in vocab, keep it. Otherwise, replace with <UNK>
        mapped_tokens = [t if t in vocab else "<UNK>" for t in tokens]
        
        # Create a copy so we don't mutate the original dictionary permanently
        new_m = m.copy()
        new_m['tokenized_code'] = ' '.join(mapped_tokens)
        mapped_dataset.append(new_m)
        
    return mapped_dataset

Before cleaning: 36499
After cleaning: 33942
After dedup: 32474

Dataset Splits:
  T1 (capped at 15000): 15000 methods
  T2 (capped at 25000): 25000 methods
  T3 (capped at 35000): 30474 methods
  Validation: 1000 methods
  Test:       1000 methods

Vocabularies Built:
  Vocab T1 size: 58206 unique tokens
  Vocab T2 size: 82923 unique tokens
  Vocab T3 size: 93365 unique tokens


---
## Save Outputs

We save:
1. `train.txt` - One tokenized method per line
2. `val.txt` - Validation set
3. `test.txt` - Test set
4. `metadata.json` - Tracks which repos/files were used (for students)

In [10]:
def save_txt(data, filename):
    """Save tokenized methods to a text file (one per line)."""
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, 'w', encoding='utf-8', errors='replace') as f:
        for method in data:
            f.write(method['tokenized_code'] + '\n')
    return filepath

# Save train/val/test files
print("Saving dataset files...\n")

# Save the three distinct training sets
train_t1_path = save_txt(T1, "train_T1.txt")
print(f"  Saved: {train_t1_path}")

train_t2_path = save_txt(T2, "train_T2.txt")
print(f"  Saved: {train_t2_path}")

train_t3_path = save_txt(T3, "train_T3.txt")
print(f"  Saved: {train_t3_path}")

# Save validation and test sets
val_path = save_txt(val_data, "val.txt")
print(f"  Saved: {val_path}")

test_path = save_txt(test_data, "test.txt")
print(f"  Saved: {test_path}")

# Save metadata
metadata = {
    "description": "Metadata for N-gram dataset. Tracks files used and exact split sizes.",
    "instructions_for_self_created_test_set": [
        "To create your self-created test set with ~1K methods:",
        "  1. Use the same repositories listed below",
        "  2. Select Java files NOT in 'selected_files'",
        "  3. Extract methods using the exact same pipeline"
    ],
    "dataset_stats": {
        "T1_size": len(T1),
        "T2_size": len(T2),
        "T3_size": len(T3),
        "val_size": len(val_data),
        "test_size": len(test_data),
        "total_repos": len(repo_java_files),
        "min_tokens": MIN_TOKENS
    },
    "repos_used": repo_java_files
}

metadata_path = os.path.join(OUTPUT_DIR, "metadata.json")
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)
print(f"  Saved: {metadata_path}")

print(f"\nAll files saved to: {OUTPUT_DIR}/")

Saving dataset files...

  Saved: dataset\ngram_dataset\train_T1.txt
  Saved: dataset\ngram_dataset\train_T2.txt
  Saved: dataset\ngram_dataset\train_T3.txt
  Saved: dataset\ngram_dataset\val.txt
  Saved: dataset\ngram_dataset\test.txt
  Saved: dataset\ngram_dataset\metadata.json

All files saved to: dataset\ngram_dataset/


---
## Summary Statistics for data pipeline and cleaning

In [ ]:
import statistics

all_token_counts = [m['token_count'] for m in tokenized_methods]

print("=" * 50)
print("         DATASET CREATION SUMMARY")
print("=" * 50)

print(f"\nRepositories:")
print(f"   Cloned:    {len(cloned_repos)}")
print(f"   Failed:    {len(failed_repos)}")

print(f"\nJava Files:")
print(f"   Selected:  {len(all_selected_files)}")

print(f"\nMethods:")
print(f"   Extracted: {stats['total']}")
print(f"   Filtered:  {stats['total'] - len(tokenized_methods)} removed")
print(f"   Final:     {len(tokenized_methods)}")

print(f"\nToken Statistics:")
print(f"   Min:    {min(all_token_counts)}")
print(f"   Max:    {max(all_token_counts)}")
print(f"   Mean:   {statistics.mean(all_token_counts):.1f}")
print(f"   Median: {statistics.median(all_token_counts):.1f}")

print(f"\nDataset Splits:")
print(f"   T1 Train:   {len(T1):,} methods")
print(f"   T2 Train:   {len(T2):,} methods")
print(f"   T3 Train:   {len(T3):,} methods")
print(f"   Validation: {len(val_data):,} methods")
print(f"   Test:       {len(test_data):,} methods")

print(f"\nOutput Files:")
print(f"   {OUTPUT_DIR}/train_T1.txt")
print(f"   {OUTPUT_DIR}/train_T2.txt")
print(f"   {OUTPUT_DIR}/train_T3.txt")
print(f"   {OUTPUT_DIR}/val.txt")
print(f"   {OUTPUT_DIR}/test.txt")
print(f"   {OUTPUT_DIR}/metadata.json")

print("\n" + "=" * 50)
print("         DATASET READY FOR N-GRAM TRAINING")
print("=" * 50)

         DATASET CREATION SUMMARY

Repositories:
   Cloned:    499
   Failed:    1

Java Files:
   Selected:  4810

Methods:
   Extracted: 39866
   Filtered:  7392 removed
   Final:     32474

Token Statistics:
   Min:    10
   Max:    33291
   Mean:   60.9
   Median: 27.0

Dataset Splits:
   T1 Train:   15,000 methods
   T2 Train:   25,000 methods
   T3 Train:   30,474 methods
   Validation: 1,000 methods
   Test:       1,000 methods

Output Files:
   dataset\ngram_dataset/train_T1.txt
   dataset\ngram_dataset/train_T2.txt
   dataset\ngram_dataset/train_T3.txt
   dataset\ngram_dataset/val.txt
   dataset\ngram_dataset/test.txt
   dataset\ngram_dataset/metadata.json

         DATASET READY FOR N-GRAM TRAINING


# N-gram Language Model

In [ ]:
import math
import json
from collections import defaultdict, Counter

class NGramLanguageModel:
    def __init__(self, n, vocab, alpha=0.01):
        """
        Initializes the N-gram model.
        :param n: The context window size (e.g., 3, 5, or 7).
        :param vocab: The vocabulary set for the training data.
        :param alpha: Smoothing parameter for Add-alpha smoothing.
        """
        self.n = n
        self.alpha = alpha
        
        # Ensure <UNK> is in the vocabulary
        self.vocab = set(vocab)
        self.vocab.add("<UNK>")
        self.vocab_size = len(self.vocab)
        
        # Data structures to store counts
        self.context_counts = defaultdict(Counter)
        self.context_totals = Counter()
        self.unigram_counts = Counter()
        
    def train(self, dataset):
        """Trains the model by calculating n-gram and context counts."""
        for method in dataset:
            tokens = method['tokenized_code'].split()
            
            # Count unigrams for fallback predictions
            for t in tokens:
                self.unigram_counts[t] += 1
                
            # Pad the beginning of the sequence to handle initial context
            padded_tokens = ["<s>"] * (self.n - 1) + tokens
            
            for i in range(len(padded_tokens) - self.n + 1):
                context = tuple(padded_tokens[i : i + self.n - 1])
                target = padded_tokens[i + self.n - 1]
                
                self.context_counts[context][target] += 1
                self.context_totals[context] += 1

    def get_probability(self, context, target):
        """
        Computes the smoothed probability P(target | context).
        """
        count_ngram = self.context_counts[context][target]
        count_context = self.context_totals[context]
        
        # Add-alpha smoothing formula
        prob = (count_ngram + self.alpha) / (count_context + self.alpha * self.vocab_size)
        return prob

    def predict_argmax(self, context):
        """
        Returns the most probable next token and its probability.
        """
        if context in self.context_counts and self.context_totals[context] > 0:
            best_token = self.context_counts[context].most_common(1)[0][0]
            best_prob = self.get_probability(context, best_token)
            return best_token, best_prob
        else:
            # Backoff: If context is entirely unseen, predict the most common unigram
            best_token = self.unigram_counts.most_common(1)[0][0]
            best_prob = self.get_probability(context, best_token)
            return best_token, best_prob

    def compute_perplexity(self, dataset):
        """
        Computes perplexity over a dataset using ground-truth probabilities.
        """
        log_prob_sum = 0.0
        N = 0
        
        for method in dataset:
            tokens = method['tokenized_code'].split()
            padded_tokens = ["<s>"] * (self.n - 1) + tokens
            
            for i in range(len(padded_tokens) - self.n + 1):
                context = tuple(padded_tokens[i : i + self.n - 1])
                target = padded_tokens[i + self.n - 1]
                
                # We use the probability of the ground-truth token, NOT the argmax prediction
                prob = self.get_probability(context, target)
                log_prob_sum += math.log(prob)
                N += 1
                
        if N == 0: return float('inf')
        
        return math.exp(-log_prob_sum / N)

In [13]:
# Group the training sets and their corresponding vocabularies
training_sets = {
    "T1": (T1, vocab_T1),
    "T2": (T2, vocab_T2),
    "T3": (T3, vocab_T3)
}

n_values = [3, 5, 7]
models = {}

best_config = None
best_perplexity = float('inf')
best_model = None
best_vocab = None

print("Training and Validating Models across T1, T2, and T3...")
print("=" * 60)

for set_name, (train_data, vocab) in training_sets.items():
    print(f"\n--- Evaluating Training Set: {set_name} ---")
    
    # Map unseen validation tokens to <UNK> using the specific vocabulary for this training set
    val_data_mapped = apply_unk_mapping(val_data, vocab)
    
    for n in n_values:
        print(f"Training {n}-gram model on {set_name}...")
        model = NGramLanguageModel(n=n, vocab=vocab, alpha=0.1)
        model.train(train_data)
        
        # Evaluate on the mapped validation set
        perplexity = model.compute_perplexity(val_data_mapped)
        print(f"  -> Perplexity for {set_name} (n={n}): {perplexity:.2f}")
        
        # Store the model using a tuple key (Dataset, n)
        models[(set_name, n)] = model
        
        # Keep track of the absolute best configuration
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_config = (set_name, n)
            best_model = model
            best_vocab = vocab

print("\n" + "=" * 60)
print(f"Best Configuration: Dataset = {best_config[0]}, n = {best_config[1]}")
print(f"Lowest Validation Perplexity: {best_perplexity:.2f}")
print("=" * 60)

Training and Validating Models across T1, T2, and T3...

--- Evaluating Training Set: T1 ---
Training 3-gram model on T1...
  -> Perplexity for T1 (n=3): 1451.24
Training 5-gram model on T1...
  -> Perplexity for T1 (n=5): 10033.38
Training 7-gram model on T1...
  -> Perplexity for T1 (n=7): 21076.88

--- Evaluating Training Set: T2 ---
Training 3-gram model on T2...
  -> Perplexity for T2 (n=3): 1341.57
Training 5-gram model on T2...
  -> Perplexity for T2 (n=5): 10302.00
Training 7-gram model on T2...
  -> Perplexity for T2 (n=7): 23986.11

--- Evaluating Training Set: T3 ---
Training 3-gram model on T3...
  -> Perplexity for T3 (n=3): 1293.73
Training 5-gram model on T3...
  -> Perplexity for T3 (n=5): 10259.14
Training 7-gram model on T3...
  -> Perplexity for T3 (n=7): 24700.76

Best Configuration: Dataset = T3, n = 3
Lowest Validation Perplexity: 1293.73


In [14]:
def generate_json_results(model, dataset, test_set_name, output_filename):
    """
    Evaluates the model on a test set and exports the detailed JSON structure.
    """
    # Compute overall perplexity first using the ground-truth probabilities
    overall_perplexity = model.compute_perplexity(dataset)
    
    results = {
        "testSet": test_set_name,
        "perplexity": round(overall_perplexity, 2),
        "data": []
    }
    
    print(f"Generating predictions for {test_set_name}...")
    
    for idx, method in enumerate(dataset):
        tokens = method['tokenized_code'].split()
        padded_tokens = ["<s>"] * (model.n - 1) + tokens
        
        predictions_list = []
        
        for i in range(len(padded_tokens) - model.n + 1):
            context = tuple(padded_tokens[i : i + model.n - 1])
            ground_truth = padded_tokens[i + model.n - 1]
            
            # For JSON output, predProbability is the max probability of the model's chosen token
            pred_token, pred_prob = model.predict_argmax(context)
            
            # Clean up padding tokens for the JSON output context array
            clean_context = [c for c in context if c != "<s>"]
            
            predictions_list.append({
                "context": clean_context,
                "predToken": pred_token,
                "predProbability": round(pred_prob, 4),
                "groundTruth": ground_truth
            })
            
        results["data"].append({
            "index": f"ID{idx + 1}",
            "tokenizedCode": method['tokenized_code'],
            "contextWindow": model.n,
            "predictions": predictions_list
        })
        
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2)
        
    print(f"  -> Saved {output_filename} (Perplexity: {overall_perplexity:.2f})")


def load_external_test_set(filepath):
    """Loads a tokenized test set from a text file into the dataset format."""
    dataset = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                dataset.append({'tokenized_code': line})
    return dataset


test_path = os.path.join(OUTPUT_DIR, "test.txt")
extract_test_data = load_external_test_set(test_path)
output_path = os.path.join(OUTPUT_DIR, "results-self-test.json")

test_data_mapped = apply_unk_mapping(extract_test_data, best_vocab)
generate_json_results(
    model=best_model, 
    dataset=test_data_mapped, 
    test_set_name="test.txt", 
    output_filename=output_path
)


given_test_path = os.path.join(OUTPUT_DIR, "given_test.txt")
extract_test_data = load_external_test_set(given_test_path)
output_path = os.path.join(OUTPUT_DIR, "results-given-test.json")

provided_test_mapped = apply_unk_mapping(extract_test_data, best_vocab)
generate_json_results(
    model=best_model, 
    dataset=provided_test_mapped, 
    test_set_name="given_test.txt", 
    output_filename=output_path
)

Generating predictions for test.txt...
  -> Saved dataset\ngram_dataset\results-self-test.json (Perplexity: 1210.13)
Generating predictions for given_test.txt...
  -> Saved dataset\ngram_dataset\results-given-test.json (Perplexity: 1864.57)
